# Baseline Experiment - Load Pre-optimized Solution

This notebook loads the best pre-optimized solution from snapshots, validates it, and computes the score.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from shapely.strtree import STRtree
from decimal import Decimal, getcontext
import json

getcontext().prec = 30

# Tree shape definition (from getting-started notebook)
def get_tree_polygon():
    """Create the Christmas tree polygon shape."""
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    
    # Vertices of the tree (15 points)
    vertices = [
        (-trunk_w/2, 0),           # trunk bottom left
        (-trunk_w/2, trunk_h),     # trunk top left
        (-base_w/2, trunk_h),      # base left
        (-mid_w/2, trunk_h + 0.2), # mid left outer
        (-base_w/2 + 0.1, trunk_h + 0.2), # mid left inner
        (-top_w/2, trunk_h + 0.4), # top left outer
        (-mid_w/2 + 0.1, trunk_h + 0.4), # top left inner
        (0, tip_y),                # tip
        (mid_w/2 - 0.1, trunk_h + 0.4), # top right inner
        (top_w/2, trunk_h + 0.4),  # top right outer
        (mid_w/2, trunk_h + 0.2),  # mid right inner
        (base_w/2 - 0.1, trunk_h + 0.2), # mid right outer
        (base_w/2, trunk_h),       # base right
        (trunk_w/2, trunk_h),      # trunk top right
        (trunk_w/2, 0),            # trunk bottom right
    ]
    return Polygon(vertices)

TREE_POLY = get_tree_polygon()
print(f"Tree polygon created with {len(TREE_POLY.exterior.coords)} vertices")
print(f"Tree bounds: {TREE_POLY.bounds}")
print(f"Tree area: {TREE_POLY.area:.6f}")

Tree polygon created with 16 vertices
Tree bounds: (-0.35, 0.0, 0.35, 0.8)
Tree area: 0.235000


In [2]:
def parse_s_value(s_val):
    """Parse a string value prefixed with 's' to float."""
    if isinstance(s_val, str) and s_val.startswith('s'):
        return float(s_val[1:])
    return float(s_val)

def load_submission(path):
    """Load submission CSV and parse values."""
    df = pd.read_csv(path)
    df['x_val'] = df['x'].apply(parse_s_value)
    df['y_val'] = df['y'].apply(parse_s_value)
    df['deg_val'] = df['deg'].apply(parse_s_value)
    
    # Extract n from id (format: NNN_T)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    df['tree_idx'] = df['id'].apply(lambda x: int(x.split('_')[1]))
    
    return df

# Load baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/submission/submission.csv'
df = load_submission(baseline_path)
print(f"Loaded {len(df)} rows")
print(f"N values range: {df['n'].min()} to {df['n'].max()}")
print(f"Expected rows: {sum(range(1, 201))} = 20100")
df.head(10)

Loaded 20100 rows
N values range: 1 to 200
Expected rows: 20100 = 20100


,id,x,y,deg,x_val,y_val,deg_val,n,tree_idx
0,001_0,s-48.196086194214246,s58.770984615214225,s45.0,-48.196086,58.770985,45.000000,1,0
1,002_0,s0.154097069621355887,s-0.038540742694794648,s203.629377730656841550,0.154097,-0.038541,203.629378,2,0
2,002_1,s-0.154097069621372845,s-0.561459257305224058,s23.629377730656791812,-0.154097,-0.561459,23.629378,2,1
3,003_0,s1.123655816140301,s0.781101815992563,s111.125132292893,1.123656,0.781102,111.125132,3,0
4,003_1,s1.23405569584216,s1.275999500663759,s66.370622269343,1.234056,1.276000,66.370622,3,1
5,003_2,s0.641714640229075,s1.180458566613381,s155.13405193710082,0.641715,1.180459,155.134052,3,2
6,004_0,s-0.324747789589372171,s0.132109978088185392,s156.370622145636389178,-0.324748,0.132110,156.370622,4,0
7,004_1,s0.315354346242637695,s0.132109978063475492,s156.370622269264089255,0.315354,0.132110,156.370622,4,1
8,004_2,s0.324747789592379210,s-0.732109978069475531,s336.370622269264003990,0.324748,-0.732110,336.370622,4,2
9,004_3,s-0.315354348134818330,s-0.732109978094185987,s336.370622145636446021,-0.315354,-0.732110,336.370622,4,3


In [3]:
def create_tree_at_position(x, y, deg):
    """Create a tree polygon at the given position and rotation."""
    # First rotate around origin, then translate
    tree = rotate(TREE_POLY, deg, origin=(0, 0))
    tree = translate(tree, x, y)
    return tree

def get_bounding_box_side(polygons):
    """Get the side length of the smallest square bounding box."""
    if not polygons:
        return 0
    
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    
    return max(width, height)

def check_overlaps(polygons):
    """Check for overlapping polygons using STRtree."""
    if len(polygons) <= 1:
        return []
    
    overlaps = []
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx > i:  # Only check each pair once
                if polygons[i].intersects(polygons[idx]):
                    if not polygons[i].touches(polygons[idx]):
                        # Check if it's a real overlap (not just touching)
                        intersection = polygons[i].intersection(polygons[idx])
                        if intersection.area > 1e-10:  # Small tolerance
                            overlaps.append((i, idx, intersection.area))
    
    return overlaps

print("Functions defined successfully")

Functions defined successfully


In [4]:
# Compute score and validate for all n
scores_by_n = {}
overlap_errors = []

for n in range(1, 201):
    n_df = df[df['n'] == n]
    
    if len(n_df) != n:
        print(f"ERROR: n={n} has {len(n_df)} trees, expected {n}")
        continue
    
    # Create polygons
    polygons = []
    for _, row in n_df.iterrows():
        poly = create_tree_at_position(row['x_val'], row['y_val'], row['deg_val'])
        polygons.append(poly)
    
    # Check for overlaps
    overlaps = check_overlaps(polygons)
    if overlaps:
        overlap_errors.append((n, overlaps))
    
    # Compute bounding box side
    side = get_bounding_box_side(polygons)
    
    # Compute score contribution: s^2 / n
    score_n = (side ** 2) / n
    scores_by_n[n] = {'side': side, 'score': score_n}
    
    if n <= 10 or n % 20 == 0:
        print(f"n={n:3d}: side={side:.6f}, score_contribution={score_n:.6f}")

print(f"\nProcessed all {len(scores_by_n)} configurations")
print(f"Overlap errors found: {len(overlap_errors)}")
if overlap_errors:
    for n, overlaps in overlap_errors[:5]:
        print(f"  n={n}: {len(overlaps)} overlaps")

n=  1: side=0.671751, score_contribution=0.451250
n=  2: side=1.109832, score_contribution=0.615863
n=  3: side=1.294276, score_contribution=0.558384
n=  4: side=1.451133, score_contribution=0.526447
n=  5: side=1.600630, score_contribution=0.512403
n=  6: side=1.708766, score_contribution=0.486647
n=  7: side=1.798154, score_contribution=0.461908
n=  8: side=1.916248, score_contribution=0.459001
n=  9: side=2.027608, score_contribution=0.456799
n= 10: side=2.099722, score_contribution=0.440883
n= 20: side=2.897863, score_contribution=0.419881


n= 40: side=3.966364, score_contribution=0.393301


n= 60: side=4.790165, score_contribution=0.382428


n= 80: side=5.339533, score_contribution=0.356383


n=100: side=5.978765, score_contribution=0.357456


n=120: side=6.448498, score_contribution=0.346526


n=140: side=7.020896, score_contribution=0.352093


n=160: side=7.492237, score_contribution=0.350835


n=180: side=7.854614, score_contribution=0.342750


n=200: side=8.349414, score_contribution=0.348564

Processed all 200 configurations
Overlap errors found: 198
  n=3: 1 overlaps
  n=4: 3 overlaps
  n=5: 2 overlaps
  n=6: 3 overlaps
  n=7: 4 overlaps


In [5]:
# Compute total score
total_score = sum(s['score'] for s in scores_by_n.values())
print(f"\n{'='*50}")
print(f"TOTAL SCORE: {total_score:.6f}")
print(f"{'='*50}")
print(f"\nTarget score: 68.888293")
print(f"Gap to target: {total_score - 68.888293:.6f}")
print(f"Percentage improvement needed: {(total_score - 68.888293) / total_score * 100:.2f}%")


TOTAL SCORE: 74.960948

Target score: 68.888293
Gap to target: 6.072655
Percentage improvement needed: 8.10%


In [6]:
# Analyze score contributions by n-range
print("\nScore breakdown by n-range:")
print("-" * 40)

ranges = [(1, 10), (11, 20), (21, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores_by_n[n]['score'] for n in range(start, end+1))
    print(f"n={start:3d}-{end:3d}: {range_score:.6f}")

print("-" * 40)
print(f"Total: {total_score:.6f}")


Score breakdown by n-range:
----------------------------------------
n=  1- 10: 4.969585
n= 11- 20: 4.235006
n= 21- 50: 11.985537
n= 51-100: 18.523315
n=101-150: 17.837792
n=151-200: 17.409713
----------------------------------------
Total: 74.960948


In [7]:
# Copy submission to output folder
import shutil

# Copy the baseline submission
shutil.copy(baseline_path, '/home/submission/submission.csv')
print("Copied baseline submission to /home/submission/submission.csv")

# Also save to experiment folder
shutil.copy(baseline_path, '/home/code/experiments/000_baseline/submission.csv')
print("Copied to experiment folder")

# Verify the copy
df_verify = pd.read_csv('/home/submission/submission.csv')
print(f"Verified: {len(df_verify)} rows in submission")

Copied baseline submission to /home/submission/submission.csv
Copied to experiment folder
Verified: 20100 rows in submission


In [8]:
# Save metrics
metrics = {
    'cv_score': total_score,
    'total_score': total_score,
    'overlap_errors': len(overlap_errors),
    'target': 68.888293,
    'gap': total_score - 68.888293
}

with open('/home/code/experiments/000_baseline/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("Saved metrics.json")
print(json.dumps(metrics, indent=2))

Saved metrics.json
{
  "cv_score": 74.96094818140274,
  "total_score": 74.96094818140274,
  "overlap_errors": 198,
  "target": 68.888293,
  "gap": 6.072655181402737
}


In [9]:
# Let's look at the overlap detection more carefully\n# The tolerance might be too strict\n\n# Check one specific overlap\nn = 3\nn_df = df[df['n'] == n]\npolygons = []\nfor _, row in n_df.iterrows():\n    poly = create_tree_at_position(row['x_val'], row['y_val'], row['deg_val'])\n    polygons.append(poly)\n\nprint(f"n={n} trees:")\nfor i, poly in enumerate(polygons):\n    print(f"  Tree {i}: bounds={poly.bounds}")\n\n# Check all pairs\nfor i in range(len(polygons)):\n    for j in range(i+1, len(polygons)):\n        if polygons[i].intersects(polygons[j]):\n            intersection = polygons[i].intersection(polygons[j])\n            print(f"  Trees {i} and {j}: intersects={polygons[i].intersects(polygons[j])}, touches={polygons[i].touches(polygons[j])}, intersection_area={intersection.area}")

In [10]:
# CORRECT tree geometry from the official getting-started notebook\n# The tree has trunk at the BOTTOM (negative y), tip at y=0.8\n\nfrom decimal import Decimal, getcontext\ngetcontext().prec = 25\nscale_factor = Decimal('1e15')\n\ndef get_correct_tree_polygon():\n    \"\"\"Create the CORRECT Christmas tree polygon shape from official notebook.\"\"\"\n    trunk_w = Decimal('0.15')\n    trunk_h = Decimal('0.2')\n    base_w = Decimal('0.7')\n    mid_w = Decimal('0.4')\n    top_w = Decimal('0.25')\n    tip_y = Decimal('0.8')\n    tier_1_y = Decimal('0.5')\n    tier_2_y = Decimal('0.25')\n    base_y = Decimal('0.0')\n    trunk_bottom_y = -trunk_h\n\n    # Vertices in the CORRECT order (starting from tip, going clockwise)\n    vertices = [\n        # Start at Tip\n        (float(Decimal('0.0')), float(tip_y)),\n        # Right side - Top Tier\n        (float(top_w / Decimal('2')), float(tier_1_y)),\n        (float(top_w / Decimal('4')), float(tier_1_y)),\n        # Right side - Middle Tier\n        (float(mid_w / Decimal('2')), float(tier_2_y)),\n        (float(mid_w / Decimal('4')), float(tier_2_y)),\n        # Right side - Bottom Tier\n        (float(base_w / Decimal('2')), float(base_y)),\n        # Right Trunk\n        (float(trunk_w / Decimal('2')), float(base_y)),\n        (float(trunk_w / Decimal('2')), float(trunk_bottom_y)),\n        # Left Trunk\n        (float(-(trunk_w / Decimal('2'))), float(trunk_bottom_y)),\n        (float(-(trunk_w / Decimal('2'))), float(base_y)),\n        # Left side - Bottom Tier\n        (float(-(base_w / Decimal('2'))), float(base_y)),\n        # Left side - Middle Tier\n        (float(-(mid_w / Decimal('4'))), float(tier_2_y)),\n        (float(-(mid_w / Decimal('2'))), float(tier_2_y)),\n        # Left side - Top Tier\n        (float(-(top_w / Decimal('4'))), float(tier_1_y)),\n        (float(-(top_w / Decimal('2'))), float(tier_1_y)),\n    ]\n    return Polygon(vertices)\n\nCORRECT_TREE = get_correct_tree_polygon()\nprint(f\"Correct tree polygon: {len(CORRECT_TREE.exterior.coords)} vertices\")\nprint(f\"Bounds: {CORRECT_TREE.bounds}\")\nprint(f\"Area: {CORRECT_TREE.area:.6f}\")\nprint(f\"\\nVertices:\")\nfor i, (x, y) in enumerate(CORRECT_TREE.exterior.coords[:-1]):\n    print(f\"  {i}: ({x:.4f}, {y:.4f})\")